<a href="https://colab.research.google.com/github/kmt19/siim_otani/blob/master/drive2gcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.9MB/s 


In [0]:
#https://drive.google.com/drive/u/2/my-drive/
dir_id = "1zHVf-O0xXgd47GaVjkpFRUuq59-4Ci5h"

# フォルダ内にあるファイル名とファイルIDを表示する
file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % dir_id})
for f in file_list :
  new_list=drive.ListFile({'q': "'%s' in parents and trashed=false" % f[0]["id"]}).GetList()
  a=drive.CreateFile({'id': new_list[0]["id"],'q': "'%s' in parents and trashed=false" % new_list[0]["id"]})
  print(a)
  a.GetContentFile('sample.dcm')
  break
!ls

GoogleDriveFile({'id': '1KAnyCwq8q8shjI_3wO2fN2P5s07kbssh', 'q': "'1KAnyCwq8q8shjI_3wO2fN2P5s07kbssh' in parents and trashed=false"})


FileNotDownloadableError: ignored

In [3]:
from google import colab
colab.drive.mount('/content/gdrive')
!ls 'gdrive/My Drive'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'Colab Notebooks'    dicom-images-train   手法のアイデア.gdoc
 dicom-images-test   train-rle.csv


In [5]:
!pip install pydicom
!git clone https://github.com/kmt19/siim_otani/
!ls

fatal: destination path 'siim_otani' already exists and is not an empty directory.
gdrive	sample_data  siim_otani


In [12]:
import os
import pydicom
from glob import glob
from tqdm import tqdm
import pandas as pd
from siim_otani.mask_functions import mask2rle,rle2mask
from siim_otani.pipeline import SIIMDataset
DRIVE_DIR = 'gdrive/My Drive/'
folders = os.listdir(DRIVE_DIR)
print(folders)


['手法のアイデア.gdoc', 'dicom-images-test', 'dicom-images-train', 'Colab Notebooks', 'train-rle.csv']


In [20]:
train_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-train')
test_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-test')
print(len(os.listdir(train_dicom_dir)))
print(len(os.listdir(test_dicom_dir)))
train_glob = f'{train_dicom_dir}/*/*/*.dcm'
test_glob = f'{test_dicom_dir}/*/*/*.dcm'

exclude_list = []
train_names = [f for f in sorted(glob(train_glob)) if f not in exclude_list]
test_names = [f for f in sorted(glob(test_glob)) if f not in exclude_list]


10712
1377


In [25]:
import pandas as pd
SEGMENTATION = DRIVE_DIR + '/train-rle.csv'
anns = pd.read_csv(SEGMENTATION)
anns.head()

"""
for folder in folders:
  new_folder = path + '/' + folder
  new_folder = new_folder + '/' + os.listdir(new_folder)[0]
  new_file = new_folder + '/' + os.listdir(new_folder)[0]
  ds = pydicom.dcmread(new_file)
  break
  """

                                             ImageId                                      EncodedPixels
0  1.2.276.0.7230010.3.1.4.8323329.5597.151787518...                                                 -1
1  1.2.276.0.7230010.3.1.4.8323329.12515.15178752...                                                 -1
2  1.2.276.0.7230010.3.1.4.8323329.4904.151787518...   175349 7 1013 12 1009 17 1005 19 1003 20 1002...
3  1.2.276.0.7230010.3.1.4.8323329.32579.15178751...   407576 2 1021 7 1015 10 1013 12 1011 14 1008 ...
4  1.2.276.0.7230010.3.1.4.8323329.32579.15178751...   252069 1 1021 3 1020 4 1018 5 1018 6 1016 7 1...


"\nfor folder in folders:\n  new_folder = path + '/' + folder\n  new_folder = new_folder + '/' + os.listdir(new_folder)[0]\n  new_file = new_folder + '/' + os.listdir(new_folder)[0]\n  ds = pydicom.dcmread(new_file)\n  break\n  "